In [1]:
import cv2
import skvideo
import math
import os
import numpy as np
cv2.__version__

'3.4.1'

In [2]:
import imageio

In [3]:
import pickle
import matplotlib.pyplot as plt
%matplotlib inline


In [4]:
def create_list_of_images(folder, max_length=20, f_lists=np.array([]), period=5):
    """
    serve to make the list of video images as a sequence with the period 
    max_length  is the length of images' sequence
    return list of images as a list of paths or as images' sequence
    """
    
    f_list = np.sort(os.listdir(folder))
    f_list = np.vectorize(lambda x: os.path.join(os.getcwd(), folder, x))(f_list) # the full names of files
    
    ind_mask = (np.arange(len(f_list)) % period == 0) # mask for images with the period
    
    # if the number of images in the folder less then max length 
    # reduce the period or fill the sequence the same images
    while (np.sum(ind_mask) < max_length) & (period > 1):
        period -=1
        ind_mask = np.arange(len(f_list)) % period == 0
        if (period ==1) & (np.sum(ind_mask) < max_length):
            f_list = np.sort(np.random.choice(f_list, size=max_length))
            return np.append(f_lists, f_list)
        
    # if the number of images in the folder  more then max length
    # images from folder are taken to create the image's sequence with the max length by shift the mask 
    for j in range(period):
        if j != 0:
            ind_mask[-1] = False
            ind_mask = np.roll(ind_mask, 1) # shift the period' mask to one step
        f_temp = f_list[ind_mask]  # images that match the period
        
        # separate the f_temp to image' sequence with the max length
        for i in range(len(f_temp) - max_length):
            _f_list = f_temp[i*max_length :(i+1)*max_length]
            if len(_f_list) == 0:
                break
            if len(_f_list) < max_length:
                _f_list = np.sort(np.random.choice(_f_list, size=max_length))
                break
            f_lists = np.append(f_lists, _f_list)
        
    return f_lists

In [9]:
def get_list_of_imagesPath(folder):
    """
    create a list of all images' paths  for all subfolders in the folder
    """

    fd_list = []
    
    if not all_dir(folder):
        return sorted(map(lambda x: os.path.join(folder, x), os.listdir(folder)))
    
    for fd in os.listdir(folder):
        path = os.path.join(os.getcwd(), folder, fd)
        fd_list += get_list_of_imagesPath(path)

    return fd_list 

In [10]:
def get_images_sequences(image_list, max_length=20, period=5):
    """
    make the list of lmage' sequences with length = max_length, 
    and use images with period in the one sequence
    """
    pass

In [7]:
def get_list_sequence_images(PATH, max_length=20, period=5):
    """
    serve to make the 2D tensor from the list of images
    with max length and period
    return 2D tensor (number of images' sequences, max length)
    """
    
    ff = np.vectorize(lambda x: os.path.join(os.getcwd(), PATH, x))(np.sort(os.listdir(PATH)))
    f_lists = np.array([])
    for f in ff:
        if len(f_lists) == 0:
            f_lists = create_list_of_images(f, max_length=max_length, period=period)
        f_lists = create_list_of_images(f, max_length=max_length, f_lists=f_lists, period=period)
    
    return f_lists.reshape(len(f_lists)/max_length, max_length)

In [8]:
f_lists = {}
for f in ['video_bad', 'video_good']:
    PATH = os.path.join('new_224_112_filter_1', f)
    f_lists[f] = get_list_sequence_images(PATH, 20) # make the dictionary of images' lists
    print PATH, f_lists[f].shape

new_224_112_filter_1/video_bad (1240, 20)
new_224_112_filter_1/video_good (1594, 20)


In [19]:
def get_image_size(folder, add_folder='train'):
    """
    return the size of images in the folder
    """
    try:
        if add_folder:
            path = os.path.join(os.getcwd(), folder, add_folder)
        else:
            path = os.path.join(os.getcwd(), folder)
        
        while os.path.isdir(path):
            filename = os.listdir(path)[0]
            path = os.path.join(path, filename)
            #print path
            
    except OSError as e:
        print e
        return

    return imageio.imread(path).shape

In [17]:
os.listdir(PATH)

['video_00001']

In [20]:
get_image_size(PATH.split('/')[0])i

(112, 224)

In [21]:
#libraries
import tensorflow as tf
import tflearn
tf.__version__

/home/oleg/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.10.0'

In [47]:
from imageio import imread

size = get_image_size(PATH.split('/')[0])

def create_tensor_from_list_of_image_sequence(image_list, size=size):
    """
    get the tensor from lists of images
    """
    # chech How much chanal does the image have?
    if len(size) == 2:
        w, h, c = size[0], size[1], 1
        X = np.full((len(image_list), w , h ), 0, dtype=np.int8)
    elif len(size) == 3:
        w, h, c = size[0], size[1], size[2]
        X = np.full((len(image_list), w , h, c), 0, dtype=np.int8)

    for i,img in enumerate(image_list):
        
        image = tf.keras.preprocessing.image.load_img(img, target_size=size)
        if c == 1:
            X[i] = tf.keras.preprocessing.image.img_to_array(image)[:,:,0]
        elif c ==3:
            X[i] = tf.keras.preprocessing.image.img_to_array(image)
         
    return X

In [32]:
img = '/home/oleg/EnkeSystem/new_224_112_filter_1/video_bad/video_00001/00001_image.jpg'
image = tf.keras.preprocessing.image.load_img(img, target_size=size)

In [33]:
imag = tf.keras.preprocessing.image.img_to_array(image)

In [34]:
imag.shape

(112, 224, 3)

In [36]:
imag[:,:,0, np.newaxis].shape

(112, 224, 1)

In [48]:
def create_tensor_of_featues(image_list, model):
    """
    from the 2D tensor of images make 3D the data's features tensor using  the model to extract image's features
    image_list - 2D tensor of images' names that were united to make images' sequences
    model - the model to extract the image' features
    return the 3D tensor (number of images, length of images' sequence, number of image' features)    
    """
    N = image_list.shape[0]
    l = image_list.shape[1]
    n,  = model.output_shape[1:]
    
    
    X = np.zeros((N, l, n))
    
    for i in range(N):
        x = create_tensor_from_list_of_image_sequence(image_list[i])
        #print x.shape
        features = model.predict_on_batch(x[:, :, :, np.newaxis])
        
        #features = tf.nn.batch_normalization(features, )
        X[i] = features.reshape(features.shape[0], features.shape[1])
        
    return X

In [38]:
cnn_model = tf.keras.models.load_model('cnn_model.h5')

In [14]:
#load json and create model
with open('cnn_model.json', 'r') as model_file:
    cnn_model = tf.keras.models.model_from_json(model_file.read())
#load weights into model
cnn_model.load_weights('cnn_model_weights.h5')

In [39]:
cnn_Model = tf.keras.Model(inputs=cnn_model.input,
                   outputs=cnn_model.get_layer(index=-3).output)

cnn_Model.output_shape

(None, 32)

In [ ]:
folders_video = ['new_224_112_filter/video_good', 'new_224_112_filter/video_bad']

In [49]:
X = create_tensor_of_featues(f_lists['video_bad'], cnn_Model)

In [50]:
X.shape

(1240, 20, 32)

In [51]:
np.save('x_bad', X)

In [52]:
X = create_tensor_of_featues(f_lists['video_good'], cnn_Model)
print X.shape
np.save('x_good', X)

(1594, 20, 32)
